In [24]:
# Importamos la librerias necesarias de selenium
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup as bs 
import requests
from selenium import webdriver # esto nos permite controlar navegadores web mediante código
from webdriver_manager.chrome import ChromeDriverManager # administra la instalación de chromedriver, que es necesario para controlar google chrome
from selenium.webdriver.chrome.service import Service # permite gestionar el servicio del driver de chrome
import time

In [25]:
# Inicializamos el driver de chrome
chromeService = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chromeService)


In [26]:
# Creamos la url de la página web que queremos scrapear
url ="https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"
url

'https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain'

In [30]:
def scrapeo_competencia(url):
    
    chromeService = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chromeService)

    driver.get(url)

    time.sleep(10)

    nombre_hotel = driver.find_elements("css selector", "div.hotelblock__content")
    estrellas_hotel = driver.find_elements("css selector", "div.hotelblock__content-ratings")
    precios_hotel = driver.find_elements("css selector", "div.rate-details__price-wrapper")

    diccionario_hoteles = {"nombre_hotel": [],
                "estrellas": [],
                "precio": []}
    
    for hotel in nombre_hotel:
        diccionario_hoteles["nombre_hotel"].append(hotel.text.split("\n")[0])
    
    for estrella in estrellas_hotel:
        diccionario_hoteles["estrellas"].append(estrella.text.split("\n")[1].strip("/"))
    
    for precio in precios_hotel:
        diccionario_hoteles["precio"].append(precio.text.split("\n")[1])

    hoteles_competencia = pd.DataFrame(diccionario_hoteles)
    hoteles_competencia["precio"] = hoteles_competencia["precio"].str.replace("€", "").astype(float)
    hoteles_competencia["estrellas"] = hoteles_competencia["estrellas"].astype(float)
    hoteles_competencia["fecha_reserva"] = pd.to_datetime("2025-02-21")
    
    hoteles_competencia.to_csv("../data/scrapeo_hoteles.csv", index = False)
    driver.close()

In [31]:
scrapeo_competencia(url)